In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

2025-06-11 05:24:23.562673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749619463.966720      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749619464.082995      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [3]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

Device set to use cuda


In [4]:
messages = [
    {"role": "user", "content": "Create a funny joke about chickens."}
]
output = pipe(messages)
print(output[0]["generated_text"])

 Why did the chicken join the band? Because it had the drumsticks!


In [5]:
# Check prompt template

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False)
prompt

'<|user|>\nCreate a funny joke about chickens.<|end|>\n<|endoftext|>'

In [7]:
output = pipe(messages, do_sample=True, temperature=1)
print(output)
print()
print(output[0]["generated_text"])

[{'generated_text': " Why don't farm animals watch sports? Because they prefer to keep their feet on the ground and won't peck at the game!"}]

 Why don't farm animals watch sports? Because they prefer to keep their feet on the ground and won't peck at the game!


In [8]:
output = pipe(messages, do_sample=True, top_p=1)
output

[{'generated_text': ' Why did the farmer put his chicken in a zoo instead of his barn?'}]

## Prompt Components

In [ ]:
persona = "Kamu adalah ahli dalam Large Language Model. Kamu unggul dalam memecah makalah rumit menjadi makalah yang mudah dimengerti.\n"
instruction = "Ringkaslah temuan utama dari makalah yang akan diberikan.\n"
context = "Rangkuman yang dibuat harus ada poin yang paling krusial yang bisa membantu peneliti mudah mengerti informasi paling vital dalam makalahnya.\n"
data_format = "Buatkan format ringkasannya dalam bentuk urutan poin-poin yang menjelaskan metodenya. Di setiap poinnya terdapat paragraf ringkas yang menjelaskan poin utamanya.\n"
audience = "Rangkuman ini ditujukan oleh peneliti sibuk yang butuh secara cepat memahami trend terbaru dalam Large Language Model.\n"
tone = "Nadanya haruslah terlihat professional dan jelas.\n"
text = "MY TEXT TO SUMMARIZE.\n"
data = f"Text to summarize: {text}"

query = persona + instruction + context + data_format + audience + tone + text + data
query

## One-shot Example

In [4]:
one_shot_prompt = [
    {
        "role": "user",
        "content": "A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word Gigamuru is:"
    },
    {
        "role": "system",
        "content": "I have a Gigamuru that my uncle gave me as a gift. I love to play it at home."
    },
    {
        "role": "user",
        "content": "To 'screeg' something is to swing a sword at it. An example of a sentence that uses the word screeg is: "
    }
]

print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))
print()

outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

## Chain of Instruction

In [ ]:
product_prompt = [
    {
        "role": "user",
        "content": "Buatkan nama dan slogan untuk sebuah chatbot yang memanfaatkan LLM."
    }
]
outputs = pipe(product_prompt)
product_description = outputs[0]["generated_text"]
product_description

In [ ]:
sales_prompt = [
    {
        "role": "user",
        "content": f"Generate sebuah sales pitch untuk produk ini: {product_description}"
    }
]
outputs = pipe(sales_prompt)
sales_pitch = outputs[0]["generated_text"]\
sales_pitch

## Chain of Tought

In [ ]:
cot_prompt = [
    {
        "role": "user",
        "content": "Si A punya 5 kelereng. Dia beli 2 bungkus kelereng, setiap bungkusnya ada 3 kelereng. Berapa banyak kelereng yang dia punya sekarang ?"
    },
    {
        "role": "assistant",
        "content": "Awalnya si A punya 5 kelereng. Setiap bungkus kelereng berisi 3 kelereng. Si A beli 2 bungkus. 5 + (3 * 2) = 11."
    },
    {
        "role": "user",
        "content": "Si A beli gorengan 3. Dia beli lagi 2 bungkus gorengan, setiap bungkusnya ada 6 gorengan. Berapa gorengan si A sekarang ?"
    }
]

outputs = pipe(cot_prompt)
outputs[0]["generated_text"]

## Zero Shot Chain of Tought

In [ ]:
zeroshot_cot_prompt = [
    {
        "role": "user",
        "content": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? Let's think step-by-step."
    }
]

outputs = pipe(zeroshot_cot_prompt)
outputs

## Zero Shot Tree of Tought

In [6]:
zeroshot_tot_prompt = [
    {
        "role": "user",
        "content": """
        Bayangkan ada 3 orang ahli matematika yang berbeda mencoba menjawab 1 pertanyaan.
        Setiap ahli tersebut haruslah menuliskan langkah per langkah bagaimana dia berpikir untuk
        menyelesaikan permasalahannya dan harus membagikan jawabannya kepada ahli yang lain.
        Diakhir para ahli harus berdiskusi dan sepakat mana jawaban yang benar.

        Berikut pertanyaannya:
        1. Berapakah hasil dari x + 5 = 7
        """
    }
]

outputs = pipe(zeroshot_tot_prompt)
outputs